In [55]:
!pip install transformers
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [57]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [58]:
import torch
import torch.nn as nn
from transformers import ElectraConfig, ElectraModel



class ELECTRALSTMClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = 'cuda'
        self.config = ElectraConfig.from_pretrained("beomi/KcELECTRA-base",
                                                    problem_type="multi_label_classification",
                                                    num_labels = 44) 
        
        self.embedding_size = 768
        self.batch_size = 32

        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base",config=self.config).to(self.device)
        self.lstm = nn.LSTM(self.embedding_size, self.embedding_size, batch_first=True, bidirectional=True).to(self.device)
        self.fc1 = nn.Linear(self.embedding_size * 5, 44)
        self.fc2 = nn.Linear(self.embedding_size * 2, 44)
        self.gelu = nn.GELU()


    def forward(self, input_ids=None, attention_mask=None, sep_idx=None):
        
        electra_output = self.electra(input_ids, attention_mask)[0]

        cls = electra_output[:, 0, :] # <CLS> embeddings
        # sep 토큰 가져오기
        sep_idx_x = sep_idx[0]
        sep_idx_y = sep_idx[1]

        idx = 0
        cnt = 0
        longest = torch.where(sep_idx_x==torch.mode(sep_idx_x).values)[0].size()[0]
        # 초기화
        sep_embeddings = torch.zeros(cls.size(0), longest, self.embedding_size).to(self.device)

        # embedding 값 집어넣어주기
        for x, y in zip(sep_idx_x, sep_idx_y):
            if idx == x:
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]
                cnt += 1
            else:
                idx += 1
                cnt = 0
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]


        # lstm 실행
        lstm_output, (h, c) = self.lstm(sep_embeddings) # (batch_size, seq_length, embedding_size)

        # lstm 처음과 끝 가져오기
        sep_first = lstm_output[:, 0, :]
        sep_last = lstm_output[:, -1, :]

        # lstm 결과와 cls 토큰 합치기
        concats = torch.cat((cls, sep_first, sep_last), dim=1)
        # fc 레이어에 넣고 44개 output
        x = self.gelu(concats)
        output = self.fc1(x)

        first_output = self.fc2(sep_first)
        last_output = self.fc2(sep_last)

        
        return output, first_output, last_output

In [59]:
def cos_similiarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1)))*np.sqrt(sum(np.square(v2))))
    similarity = dot_product/l2_norm

    return similarity

In [60]:
import kss

def kss_sentence(sent):
    x = ''
    split_sent = kss.split_sentences(sent)
    for i,s in enumerate(split_sent):
        if i == 0:
            x = s
        else:
            x += ' [SEP] ' + s
    return x

In [61]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base', do_lower_case=False)

def embedding(text):
    embeddings = tokenizer(text,
                           truncation=True,
                           max_length=512,
                           padding="max_length",
                           return_token_type_ids=False,
                           return_attention_mask=True,
                           add_special_tokens=True)
    return embeddings

In [62]:
dic_ori = "졸업식이 취소되어서 너무 아쉽다. 취소 이유는 중국 우한에서 처음 발병한 신종 코로나 바이러스 때문이다. 인생 마지막 졸업식인데 이렇게 취소가 되어버리다니 너무 허무하다. 사실은 졸업식에 별로 가고 싶지 않았다. 강의실에 우르르 모여 앉아 형식적인 학생회장의 말을 듣고, 그저 강의하는 사람 그 이상 그 이하도 아니었던 교수님들과 어색한 포옹을 하고 악수를 하며 학위기를 받는 그 상황이 너무 민망하고 어색할 것 같아서 가고 싶은 마음이 없었다. 그러다 마음을 바꾸게 된 건 우리 엄마의 부탁 때문이었다. 원래대로라면 2년 전에 이미 졸업을 했었어야 하지만 휴학, 유예 등으로 졸업이 2년이나 밀렸다. 2년을 기다려주시고, 이제 50대가 된 우리 엄마가 '더 늙기 전에 학사모 쓰고 사진 찍고 싶어!'라며 귀여운 부탁을 했다. 끝까지 거절하려다가 정말 얼마전에 엄마한테 졸업식 가자고 말씀드렸고, 엄마도 졸업식 날에 맞춰 연차까지 쓰셨지만 타이밍도 참. 속상하다. 팔정도에 있는 코끼리는 졸업식 때에만 올라탈 수 있다는 소문이 있다. 처음엔 갈 생각 없었던 졸업식이지만, 갈 마음을 먹고 나니 학사모 쓰고 코끼리 올라 탈 생각에 설레었는데 졸업식이 아예 취소가 되어버리니까 아쉬운 마음이 드는 건 왜일까? 8월에 있을 가을 학위수여식과 함께 진행한다고 하는데, 어느 누가 그 때를 기억하고 가겠어. 나는 괜찮은데, 엄마가 너무 아쉬워해서 마음이 쓰인다. 대학 입학부터 졸업까지 아무탈 없이, 걱정 없이 학교 생활 할 수 있게 도와주신 부모님께 졸업식 날에 맞춰서 꽃다발 선물이라도 해드려야겠다는 생각을 했다. 부모님 덕분에 좋은 학교 다니며, 더 멋진 사람으로 성장할 수 있게 도와주신 것에 대한 감사의 표현을 하는 것은 당연한 것 아닌가? '조금만 더 열심히 할 걸' 아쉬움이 남는 4년의 학교생활이었지만, 부모님 곁을 떠나 처음으로 타지에서 혼자 생활을 해보는 것부터 시작해서 처음 경험해볼 수 있었던 것들이 많아 행복했다. 물론 행복만 한 건 아니었지만, 대학 생활이 좋은 기억으로만 남지는 않겠지만, 그래도 소중했던 4년의 시간 덕분에 스스로 성장할 수 있었다."
dic_data = kss_sentence(dic_ori)


movie_path = '/content/drive/MyDrive/final_project/영화데이터.xlsx'

movie_ori = pd.read_excel(movie_path)

movie_data = []
for n in range(10):
    movie_data.append(kss_sentence(movie_ori['줄거리'][n]))

dic_emb = embedding(dic_data)

movie_emb = []
for i in range(10):
    movie_emb.append(embedding(movie_data[i]))

In [63]:
PATH = '/content/drive/MyDrive/final_project/best_model.pth'

model = ELECTRALSTMClassification()
model.load_state_dict(torch.load(PATH),strict=False)
model.to('cuda')
model.eval()

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ELECTRALSTMClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,)

In [64]:
input_id = torch.LongTensor(dic_emb['input_ids']).unsqueeze(0).to('cuda')
mask = torch.LongTensor(dic_emb['attention_mask']).unsqueeze(0).to('cuda')
sep_idx = torch.where(input_id == 3)
 
y_pred = model(input_id, mask, sep_idx)

In [65]:
for i in range(44):
    print(y_pred[0].to('cpu').tolist()[0][i])
    if y_pred[0].to('cpu').tolist()[0][i] >= 0.3:
        print(i)

-0.04543263837695122
-0.0105989845469594
0.030848342925310135
-0.004434601403772831
-0.06430709362030029
-0.09149645268917084
0.09236409515142441
-0.030949797481298447
0.02501162514090538
0.02429257333278656
-0.0008861669339239597
0.05687211453914642
0.016009172424674034
-0.024489164352416992
0.1302417516708374
-0.06370110809803009
-0.11530178785324097
0.034983981400728226
0.058630917221307755
-0.015609865076839924
-0.021420519798994064
0.006686437409371138
-0.08734215050935745
-0.03961938992142677
0.08860105276107788
0.027330301702022552
-0.012510018423199654
0.08345218747854233
0.039134759455919266
-0.07578951120376587
-0.06853248178958893
0.04582741856575012
-0.05140140280127525
-0.04644513130187988
-0.013921678066253662
-0.009487727656960487
-0.06807570159435272
-0.03653861582279205
-0.057973865419626236
-0.0006428617052733898
0.0108445193618536
0.036166682839393616
0.030680039897561073
0.057493049651384354
